# Question: Does the company treat its employees well? 

This section relates to cultural aspects of the companies being examined, including the chief concern: how employees are treated. We also consider worker safety; fair pay and benefits; opportunities for development, training, and advancement; and other aspects that impact the company's workers.

Employees are the lifeblood of a company. Happy, healthy, and valued employees are more willing and able to do higher-quality work, be enthusiastic brand evangelists, unleash their creativity to invent better services and solutions, and innovate to improve the company. Employees should be viewed as valuable assets to invest in continually, not expendable "resources" or drags on the bottom line. (The "bottom line" is net profit -- a company's income after all expenses have been deducted from revenue.)

Companies that excel at engaging their employees actually achieve per-share earnings growth more than four times that of their rivals, according to Gallup. Compared to companies in the bottom quartile, the top-quartile companies (based on employee engagement) generate higher customer engagement, higher productivity, better retention, fewer accidents, and 21% higher profitability.

Even though Wall Street tends to cheer when companies lay off workers, high employee turnover is actually an expense to be avoided. Not only is it a financial cost -- think about severance packages, and the costs of recruiting and training new employees as well as retraining remaining workers -- but the loss of intellectual capital is also a poor outcome for employers.

Company websites and sustainability reports can help you assess this factor. Also look for publications from organizations that rate companies on worker treatment, such as Fortune's annual list of "100 Best Companies to Work For" and Forbes' "Just 100" list.

We also look for negative elements, like shoddy employment treatment; contentious relationships with unions; lawsuits or controversies about discrimination; harassment or wage theft; and other behavior that indicates poor employee relations, like serial layoffs or constant restructuring. These kinds of red flags might disqualify a company for inclusion in our ESG portfolio.

# Data Sources

Possible Sites to Aggregate Employee Reviews:
- Glassdoor 
- Indeed
- Vault 
- CareerBliss
- Kununu
- JobAdvisor
- Ratemyemployer
- TheJobCrowd
- LookBeforeYouLeap
- Comparably
- Yelp

# Imports

In [1]:
# All necessary imports
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
import pandas as pd
import pickle

# Indeed

In [19]:
companies = ['Apple','Microsoft']

In [20]:
def get_indeed_reviews(companies):
    
    # Create empty dataframe
    indeed_reviews = pd.DataFrame(columns=['company','work_life_balance','pay_and_benefits','job_security_and_advancement','management','culture','composite_score'])
    
    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # # Define driver
    # driver = webdriver.Chrome(PATH)
    driver.set_window_size(1080,800)
    # Define url
    url= "https://www.indeed.com/?from=gnav-acme--acme-webapp"
    # Get website
    driver.get(url)
    
    for company in companies:
        # Find search bar
        search_bar = driver.find_element_by_xpath('//*[@id="text-input-what"]')
        # Clear search bar
        search_bar.clear()
        # Enter company name into search bar
        search_bar.send_keys(company)
        # Search company
        search_bar.send_keys(Keys.ENTER)
        time.sleep(3)
        try:
            exit_button = driver.find_element_by_xpath('//*[@id="popover-x"]/button')
            exit_button.click()
        except:
            pass
        # Find Reviews
        driver.find_element_by_link_text('Reviews').click()
        # Define variables for employee ratings
        work_life_balance = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[1]/a/span[1]').text)
        pay_and_benefits = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[2]/a/span[1]').text)
        job_security_and_advancement = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[3]/a/span[1]').text)
        management = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[4]/a/span[1]').text)
        culture = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[5]/a/span[1]').text)
        composite_score = (work_life_balance+pay_and_benefits+job_security_and_advancement+management+culture) / 25
        
        # Append new data to dataframe
        indeed_reviews = indeed_reviews.append({'company': company,
                                                'work_life_balance': work_life_balance,
                                                'pay_and_benefits': pay_and_benefits,
                                                'job_security_and_advancement': job_security_and_advancement,
                                                'management': management,
                                                'culture': culture,
                                                'composite_score': composite_score}, ignore_index=True)
        
        time.sleep(1)
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(1)
    
    return indeed_reviews
    

In [21]:
get_indeed_reviews(companies)

,company,work_life_balance,pay_and_benefits,job_security_and_advancement,management,culture,composite_score
0,Apple,3.8,4.1,3.7,3.7,4.1,0.776
1,Microsoft,3.9,4.1,3.5,3.6,4.0,0.764
